In [494]:
import json
import pandas as pd

In [495]:
heart = pd.read_json('~/Desktop/py129/self-viz/sample_heart.json')

# Empty lists for heartbeats/min and Fitbit 'Confidence' value
bpm_list = []
conf_list = []

# Populates empty lists with respective Fitbit values
for i in heart['value']:
    bpm_list.append(i['bpm'])
    conf_list.append(i['confidence'])

# Creates new pandas dataframes from populated lists
bpm_df = pd.DataFrame(bpm_list)
conf_df = pd.DataFrame(conf_list)

# Assigns dataframes as new series/columns in original dataframe
heart['BPM'] = bpm_df
heart['Confidence'] = conf_df

# Exports dataframe as CSV
heart[['dateTime','BPM','Confidence']].to_csv(('~/Desktop/py129/self-viz/sample_heart.csv'))


In [496]:
# Reads JSON file and renames column for Calories burned per minute
calor = pd.read_json('~/Desktop/py129/self-viz/sample_calories.json')
calor.rename(columns={'value':'Calories'}, inplace=True)

# Exports dataframe as CSV
calor.to_csv(('~/Desktop/py129/self-viz/sample_calories.csv'))


In [497]:
heart = pd.read_csv('~/Desktop/py129/self-viz/denver_trip.csv')

In [498]:
import sqlite3

try:
    # Creates connection to new DB, will create DB if nonexistant
    conn = sqlite3.connect('Heart-Calories-Trip')
    
    
    # Cursor will give us result set
    cursor = conn.cursor()
    print("Connected")
    
    hearttable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        heart(
                            time_id INTEGER PRIMARY KEY,
                            dateTime INTEGER NOT NULL,
                            Bpm INTEGER NOT NULL,
                            Confidence INTEGER NOT NULL
                        )
                '''
    cursor.execute(hearttable_sql)
    conn.commit()
    print("Created HEART Table")
    
    calorietable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        calorie(
                            time_id INTEGER PRIMARY KEY,
                            dateTime INTEGER NOT NULL,
                            Calories INTEGER NOT NULL
                        )
                '''
    cursor.execute(calorietable_sql)
    conn.commit()
    print("Created CALORIE Table")

    triptable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        trip(
                            time_id INTEGER PRIMARY KEY,
                            name TEXT,
                            address TEXT,
                            description TEXT,
                            TimeSpan_begin INTEGER NOT NULL,
                            TimeSpan_end INTEGER NOT NULL,
                            longitude INTEGER,
                            latitude INTEGER,
                            LineString_coordinates TEXT
                        )
                '''
    cursor.execute(triptable_sql)
    conn.commit()
    print("Created TRIP Table")
    
except sqlite3.Error as error:
        print("Error")
        print(error)
# Always close up our resources
finally:
    if cursor:
        cursor.close()
        if conn:
            conn.close()

Connected
Created HEART Table
Created CALORIE Table
Created TRIP Table


In [499]:
def close_DB_resources(dbconn, cursor):
    try:
        if dbconn:
            dbconn.close()
        if cursor:
            cursor.close()
        print("closed resources")
    except sqlite3.Error as err:
        ("Error closing resources")

In [500]:
# Utility for handling DB errors
def handle_DB_error(dbconn, cursor):
    if dbconn:
        try:
            dbconn.rollback()
            print("Rolled back transation")
        except sqlite3.Error as error:
            print("Error rolling back transaction")
        finally:
            close_DB_resources(dbconn, cursor)
            dbconn = None
            return dbconn

In [501]:
# Utility DB connection and cursor function to return our access objects
def connect_SQLite():
    try:
        dbconn = sqlite3.connect('Heart-Calories-Trip')
        cursor = dbconn.cursor()
        print("Connected to Heart-Calories-Trip")
    except sqlite3.Error as error:
        print("Error opening database")
        dbconn = handle_DB_error(dbconn, cursor)
    return dbconn, cursor

In [502]:
import csv

heart_insert = '''INSERT INTO heart (time_id, dateTime, Bpm, Confidence)
                  VALUES (?,?,?,?);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('sample_heart.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['BPM'], time['Confidence'])
        print(record)
        cursor.execute(heart_insert, record)
        dbconn.commit()

close_DB_resources(dbconn, cursor)

Connected to Heart-Calories-Trip
(1, '2020-10-04 06:00:01', '130', '1')
(2, '2020-10-04 06:00:06', '132', '1')
(3, '2020-10-04 06:00:21', '133', '1')
(4, '2020-10-04 06:00:31', '132', '2')
(5, '2020-10-04 06:00:41', '131', '2')
(6, '2020-10-04 06:00:46', '129', '1')
(7, '2020-10-04 06:00:51', '130', '1')
(8, '2020-10-04 06:00:56', '132', '2')
(9, '2020-10-04 06:01:01', '130', '2')
(10, '2020-10-04 06:01:11', '128', '2')
(11, '2020-10-04 06:01:16', '126', '2')
(12, '2020-10-04 06:01:31', '127', '2')
(13, '2020-10-04 06:01:36', '130', '2')
(14, '2020-10-04 06:01:41', '131', '2')
(15, '2020-10-04 06:01:46', '130', '2')
(16, '2020-10-04 06:01:51', '129', '2')
(17, '2020-10-04 06:01:56', '130', '2')
(18, '2020-10-04 06:02:01', '129', '2')
(19, '2020-10-04 06:02:06', '127', '1')
(20, '2020-10-04 06:02:11', '126', '1')
(21, '2020-10-04 06:02:16', '125', '1')
(22, '2020-10-04 06:02:21', '125', '2')
(23, '2020-10-04 06:02:26', '126', '2')
(24, '2020-10-04 06:02:31', '127', '2')
(25, '2020-10-04

(204, '2020-10-04 06:23:45', '111', '2')
(205, '2020-10-04 06:23:50', '110', '2')
(206, '2020-10-04 06:24:00', '113', '2')
(207, '2020-10-04 06:24:05', '102', '3')
(208, '2020-10-04 06:24:10', '94', '3')
(209, '2020-10-04 06:24:15', '97', '3')
(210, '2020-10-04 06:24:30', '97', '3')
(211, '2020-10-04 06:24:35', '98', '3')
(212, '2020-10-04 06:24:40', '99', '2')
(213, '2020-10-04 06:24:50', '100', '2')
(214, '2020-10-04 06:24:55', '101', '2')
(215, '2020-10-04 06:25:00', '110', '3')
(216, '2020-10-04 06:25:05', '105', '3')
(217, '2020-10-04 06:25:10', '101', '3')
(218, '2020-10-04 06:25:15', '100', '2')
(219, '2020-10-04 06:25:25', '101', '2')
(220, '2020-10-04 06:25:30', '102', '2')
(221, '2020-10-04 06:25:35', '108', '2')
(222, '2020-10-04 06:25:40', '107', '2')
(223, '2020-10-04 06:25:50', '103', '3')
(224, '2020-10-04 06:25:55', '101', '3')
(225, '2020-10-04 06:26:00', '101', '2')
(226, '2020-10-04 06:26:05', '102', '2')
(227, '2020-10-04 06:26:15', '104', '2')
(228, '2020-10-04 06:

(483, '2020-10-04 07:01:05', '79', '3')
(484, '2020-10-04 07:01:10', '81', '3')
(485, '2020-10-04 07:01:15', '83', '3')
(486, '2020-10-04 07:01:25', '80', '3')
(487, '2020-10-04 07:01:40', '80', '3')
(488, '2020-10-04 07:01:45', '79', '3')
(489, '2020-10-04 07:01:55', '80', '3')
(490, '2020-10-04 07:02:10', '80', '3')
(491, '2020-10-04 07:02:15', '81', '3')
(492, '2020-10-04 07:02:25', '80', '3')
(493, '2020-10-04 07:02:35', '79', '3')
(494, '2020-10-04 07:02:50', '79', '3')
(495, '2020-10-04 07:03:00', '80', '3')
(496, '2020-10-04 07:03:10', '82', '3')
(497, '2020-10-04 07:03:16', '84', '3')
(498, '2020-10-04 07:03:21', '83', '3')
(499, '2020-10-04 07:03:26', '82', '3')
(500, '2020-10-04 07:03:31', '81', '3')
(501, '2020-10-04 07:03:36', '80', '3')
(502, '2020-10-04 07:03:51', '81', '3')
(503, '2020-10-04 07:03:56', '80', '3')
(504, '2020-10-04 07:04:06', '82', '3')
(505, '2020-10-04 07:04:11', '84', '3')
(506, '2020-10-04 07:04:16', '85', '3')
(507, '2020-10-04 07:04:21', '84', '3')


(815, '2020-10-04 07:46:13', '94', '3')
(816, '2020-10-04 07:46:18', '97', '3')
(817, '2020-10-04 07:46:23', '98', '3')
(818, '2020-10-04 07:46:28', '97', '3')
(819, '2020-10-04 07:46:33', '94', '3')
(820, '2020-10-04 07:46:38', '90', '3')
(821, '2020-10-04 07:46:43', '86', '3')
(822, '2020-10-04 07:46:48', '85', '3')
(823, '2020-10-04 07:46:53', '86', '3')
(824, '2020-10-04 07:46:58', '85', '3')
(825, '2020-10-04 07:47:03', '82', '3')
(826, '2020-10-04 07:47:08', '81', '3')
(827, '2020-10-04 07:47:23', '81', '3')
(828, '2020-10-04 07:47:28', '83', '3')
(829, '2020-10-04 07:47:43', '83', '3')
(830, '2020-10-04 07:47:53', '85', '3')
(831, '2020-10-04 07:47:58', '87', '3')
(832, '2020-10-04 07:48:03', '86', '3')
(833, '2020-10-04 07:48:08', '84', '3')
(834, '2020-10-04 07:48:13', '83', '3')
(835, '2020-10-04 07:48:18', '84', '3')
(836, '2020-10-04 07:48:23', '87', '3')
(837, '2020-10-04 07:48:28', '89', '3')
(838, '2020-10-04 07:48:33', '88', '3')
(839, '2020-10-04 07:48:38', '86', '3')


(1030, '2020-10-04 08:15:54', '86', '3')
(1031, '2020-10-04 08:15:59', '85', '3')
(1032, '2020-10-04 08:16:04', '84', '3')
(1033, '2020-10-04 08:16:14', '85', '3')
(1034, '2020-10-04 08:16:19', '84', '3')
(1035, '2020-10-04 08:16:24', '83', '3')
(1036, '2020-10-04 08:16:34', '84', '3')
(1037, '2020-10-04 08:16:39', '83', '3')
(1038, '2020-10-04 08:16:49', '84', '3')
(1039, '2020-10-04 08:16:54', '85', '3')
(1040, '2020-10-04 08:17:09', '85', '3')
(1041, '2020-10-04 08:17:14', '84', '3')
(1042, '2020-10-04 08:17:29', '84', '3')
(1043, '2020-10-04 08:17:34', '86', '3')
(1044, '2020-10-04 08:17:39', '85', '3')
(1045, '2020-10-04 08:17:49', '86', '3')
(1046, '2020-10-04 08:18:04', '86', '3')
(1047, '2020-10-04 08:18:09', '84', '3')
(1048, '2020-10-04 08:18:14', '83', '3')
(1049, '2020-10-04 08:18:29', '83', '3')
(1050, '2020-10-04 08:18:39', '85', '3')
(1051, '2020-10-04 08:18:49', '87', '3')
(1052, '2020-10-04 08:18:54', '88', '3')
(1053, '2020-10-04 08:18:59', '87', '3')
(1054, '2020-10-

(1256, '2020-10-04 08:45:23', '85', '3')
(1257, '2020-10-04 08:45:28', '83', '3')
(1258, '2020-10-04 08:45:38', '84', '3')
(1259, '2020-10-04 08:45:43', '83', '3')
(1260, '2020-10-04 08:45:48', '82', '3')
(1261, '2020-10-04 08:45:53', '81', '3')
(1262, '2020-10-04 08:45:58', '80', '3')
(1263, '2020-10-04 08:46:03', '79', '3')
(1264, '2020-10-04 08:46:08', '80', '3')
(1265, '2020-10-04 08:46:13', '81', '3')
(1266, '2020-10-04 08:46:18', '80', '3')
(1267, '2020-10-04 08:46:33', '82', '3')
(1268, '2020-10-04 08:46:38', '82', '2')
(1269, '2020-10-04 08:46:48', '88', '2')
(1270, '2020-10-04 08:46:53', '93', '2')
(1271, '2020-10-04 08:46:58', '91', '2')
(1272, '2020-10-04 08:47:03', '88', '3')
(1273, '2020-10-04 08:47:08', '87', '3')
(1274, '2020-10-04 08:47:13', '85', '3')
(1275, '2020-10-04 08:47:18', '83', '3')
(1276, '2020-10-04 08:47:23', '82', '3')
(1277, '2020-10-04 08:47:28', '81', '3')
(1278, '2020-10-04 08:47:33', '80', '3')
(1279, '2020-10-04 08:47:38', '78', '3')
(1280, '2020-10-

(1486, '2020-10-04 09:19:48', '97', '2')
(1487, '2020-10-04 09:19:53', '92', '2')
(1488, '2020-10-04 09:19:58', '86', '3')
(1489, '2020-10-04 09:20:03', '82', '3')
(1490, '2020-10-04 09:20:08', '79', '3')
(1491, '2020-10-04 09:20:13', '77', '3')
(1492, '2020-10-04 09:20:18', '76', '3')
(1493, '2020-10-04 09:20:28', '75', '3')
(1494, '2020-10-04 09:20:33', '76', '3')
(1495, '2020-10-04 09:20:38', '75', '3')
(1496, '2020-10-04 09:20:48', '77', '3')
(1497, '2020-10-04 09:20:53', '79', '3')
(1498, '2020-10-04 09:20:58', '81', '3')
(1499, '2020-10-04 09:21:03', '82', '3')
(1500, '2020-10-04 09:21:08', '84', '3')
(1501, '2020-10-04 09:21:14', '88', '3')
(1502, '2020-10-04 09:21:19', '92', '3')
(1503, '2020-10-04 09:21:24', '95', '3')
(1504, '2020-10-04 09:21:29', '96', '3')
(1505, '2020-10-04 09:21:34', '93', '3')
(1506, '2020-10-04 09:21:39', '88', '3')
(1507, '2020-10-04 09:21:44', '85', '3')
(1508, '2020-10-04 09:21:49', '84', '3')
(1509, '2020-10-04 09:21:54', '87', '3')
(1510, '2020-10-

(1765, '2020-10-04 09:56:41', '79', '3')
(1766, '2020-10-04 09:56:46', '80', '3')
(1767, '2020-10-04 09:56:51', '81', '3')
(1768, '2020-10-04 09:56:56', '82', '3')
(1769, '2020-10-04 09:57:01', '80', '3')
(1770, '2020-10-04 09:57:06', '79', '3')
(1771, '2020-10-04 09:57:21', '79', '3')
(1772, '2020-10-04 09:57:31', '78', '3')
(1773, '2020-10-04 09:57:41', '79', '3')
(1774, '2020-10-04 09:57:46', '78', '3')
(1775, '2020-10-04 09:57:56', '80', '3')
(1776, '2020-10-04 09:58:01', '81', '3')
(1777, '2020-10-04 09:58:11', '80', '3')
(1778, '2020-10-04 09:58:21', '81', '3')
(1779, '2020-10-04 09:58:31', '80', '3')
(1780, '2020-10-04 09:58:41', '78', '3')
(1781, '2020-10-04 09:58:56', '79', '3')
(1782, '2020-10-04 09:59:01', '80', '3')
(1783, '2020-10-04 09:59:11', '81', '3')
(1784, '2020-10-04 09:59:26', '81', '3')
(1785, '2020-10-04 09:59:41', '80', '3')
(1786, '2020-10-04 09:59:46', '79', '3')


In [503]:
calorie_insert = '''INSERT INTO calorie (time_id, dateTime, Calories)
                    VALUES (?,?,?);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('sample_calories.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['Calories'])
        print(record)
        cursor.execute(calorie_insert, record)
        dbconn.commit()
        
close_DB_resources(dbconn, cursor)

Connected to Heart-Calories-Trip
(1, '2020-10-04 06:00:00', '4.72')
(2, '2020-10-04 06:01:00', '5.66')
(3, '2020-10-04 06:02:00', '5.19')
(4, '2020-10-04 06:03:00', '1.18')
(5, '2020-10-04 06:04:00', '1.18')
(6, '2020-10-04 06:05:00', '1.18')
(7, '2020-10-04 06:06:00', '1.18')
(8, '2020-10-04 06:07:00', '1.18')
(9, '2020-10-04 06:08:00', '1.18')
(10, '2020-10-04 06:09:00', '1.42')
(11, '2020-10-04 06:10:00', '1.18')
(12, '2020-10-04 06:11:00', '1.42')
(13, '2020-10-04 06:12:00', '1.18')
(14, '2020-10-04 06:13:00', '1.18')
(15, '2020-10-04 06:14:00', '1.42')
(16, '2020-10-04 06:15:00', '1.18')
(17, '2020-10-04 06:16:00', '1.18')
(18, '2020-10-04 06:17:00', '1.18')
(19, '2020-10-04 06:18:00', '1.18')
(20, '2020-10-04 06:19:00', '1.18')
(21, '2020-10-04 06:20:00', '1.42')
(22, '2020-10-04 06:21:00', '1.18')
(23, '2020-10-04 06:22:00', '1.18')
(24, '2020-10-04 06:23:00', '1.18')
(25, '2020-10-04 06:24:00', '1.18')
(26, '2020-10-04 06:25:00', '1.18')
(27, '2020-10-04 06:26:00', '1.18')
(28,

(239, '2020-10-04 09:58:00', '1.53')
(240, '2020-10-04 09:59:00', '1.42')


In [504]:
trip_insert = '''INSERT INTO trip (time_id, name, address, description, TimeSpan_begin, TimeSpan_end, longitude, latitude, LineString_coordinates)
                 VALUES (?,?,?,?,?,?,?,?,?);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('denver_trip.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['\ufeffname'], time['address'], time['description'], time['TimeSpan_begin'], time['TimeSpan_end'], time['longitude'], time['latitude'], time['LineString_coordinates'])
        print(record)
        cursor.execute(trip_insert, record)
        dbconn.commit()
        
close_DB_resources(dbconn, cursor)

Connected to Heart-Calories-Trip
(1, 'Home (701 Roosevelt Blvd)', '701 Roosevelt Blvd, Freedom, PA 15042', 'Building from 2020-10-03T00:21:12.327Z to 2020-10-03T08:22:40.539Z. Distance 0m', '2020-10-03T00:21:12.327Z', '2020-10-03T08:22:40.539Z', '-80.1614584', '40.6882963', '')
(2, 'Driving', '', 'Driving from 2020-10-03T08:22:40.539Z to 2020-10-03T09:01:36.000Z. Distance 40103m', '2020-10-03T08:22:40.539Z', '2020-10-03T09:01:36.000Z', '', '', '-80.1549814,40.6805847,0 -80.15042449999999,40.675959,0 -80.15022799999998,40.6753216,0 -80.1504375,40.6726753,0 -80.15032459999999,40.672378900000005,0 -80.14822549999998,40.6701755,0 -80.14629469999997,40.6697043,0 -80.14367189999999,40.66832840000001,0 -80.14113989999998,40.6661973,0 -80.13802209999997,40.6650261,0 -80.13633499999997,40.6646723,0 -80.13396769999997,40.6634572,0 -80.13176979999997,40.6619856,0 -80.13026189999998,40.6617219,0 -80.12906849999997,40.661219200000005,0 -80.12832789999997,40.6607311,0 -80.12374959999997,40.6607824,0

(131, 'Flying', '', 'Flying from 2020-10-11T23:10:58.000Z to 2020-10-12T01:57:15.000Z. Distance 2071886m', '2020-10-11T23:10:58.000Z', '2020-10-12T01:57:15.000Z', '', '', '-104.6737376,39.8560963,0 -104.6737376,39.8560963,0 -104.6775163,39.8622727,0 -80.2352233,40.491920799999995,0 -80.2352233,40.491920799999995,0')
(132, 'Pittsburgh International Airport', '1000 Airport Blvd, Pittsburgh, PA 15231', 'Airport from 2020-10-12T01:57:15.000Z to 2020-10-12T03:05:00.000Z. Distance 0m', '2020-10-12T01:57:15.000Z', '2020-10-12T03:05:00.000Z', '-80.2352233', '40.4919208', '')
(133, 'Driving', '', 'Driving from 2020-10-12T03:05:00.000Z to 2020-10-12T03:53:10.000Z. Distance 22706m', '2020-10-12T03:05:00.000Z', '2020-10-12T03:53:10.000Z', '', '', '-80.26107189999999,40.4901714,0 -80.25893169999999,40.493029899999996,0 -80.25871159999998,40.4936343,0 -80.2589855,40.4953043,0 -80.2588316,40.4981441,0 -80.2592205,40.4988121,0 -80.25966569999999,40.4990509,0 -80.26364809999998,40.4993484,0 -80.2650427

In [506]:
# Verify that we've got data

select_sql = '''
                SELECT *
                    FROM trip
                    WHERE address = '1441 Little Raven St, Denver, CO 80202';
            '''

dbconn, cursor = connect_SQLite()
cursor.execute(select_sql)
rec = cursor.fetchall()
for r in rec:
    print(r)

close_DB_resources(dbconn, cursor)

Connected to Heart-Calories-Trip
(23, 'The Confluence', '1441 Little Raven St, Denver, CO 80202', 'Apartment building from 2020-10-03T22:17:09.000Z to 2020-10-04T01:05:46.000Z. Distance 0m', '2020-10-03T22:17:09.000Z', '2020-10-04T01:05:46.000Z', -105.0072998, 39.7542338, '')
(24, 'The Confluence', '1441 Little Raven St, Denver, CO 80202', 'Apartment building from 2020-10-03T22:17:09.000Z to 2020-10-04T01:05:46.000Z. Distance 0m', '2020-10-03T22:17:09.000Z', '2020-10-04T01:05:46.000Z', -105.0072998, 39.7542338, '')
(28, 'The Confluence', '1441 Little Raven St, Denver, CO 80202', 'Apartment building from 2020-10-04T05:17:05.000Z to 2020-10-04T15:43:38.000Z. Distance 0m', '2020-10-04T05:17:05.000Z', '2020-10-04T15:43:38.000Z', -105.0072998, 39.7542338, '')
(29, 'The Confluence', '1441 Little Raven St, Denver, CO 80202', 'Apartment building from 2020-10-04T05:17:05.000Z to 2020-10-04T15:43:38.000Z. Distance 0m', '2020-10-04T05:17:05.000Z', '2020-10-04T15:43:38.000Z', -105.0072998, 39.7542